In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [ ]:
import minsearch
import json
import requests 

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section']
    , keyword_fields=['course']
)

In [ ]:
index.fit(documents)

In [ ]:
q = 'is it possible to get a certificate if I joined later?'

In [ ]:
boost = {
    'question': 3.0
}

results = index.search(
    query=q
    , boost_dict=boost
    , filter_dict={ 'course': 'data-engineering-zoomcamp' }
    , num_results=5
)

In [ ]:
from openai import OpenAI


In [ ]:
client = OpenAI()

In [ ]:
q

In [ ]:
response = client.chat.completitions.create(
    model="gpt-4o"
    , messages=[{"role": "user", "content": q}]
)

In [ ]:
response.choises[0].message.content

In [ ]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [ ]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"

In [ ]:
print(context)

In [ ]:
prompt=prompt_template.format(question=q, context=context).strip()

In [ ]:
response = client.chat.completitions.create(
    model="gpt-4o"
    , messages=[{"role": "user", "content": prompt}]
)